# Tutorial 1: Calculando ENSO index com o xarray

**Autor:** Elisa Passos

## Importanto as bibliotecas

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Polygon
import numpy as np
from datetime import datetime
from pythia_datasets import DATASETS
from dateutil.relativedelta import relativedelta
import pandas as pd
import xarray as xr
from noaa_ftp import NOAA

# Objetivos do Tutorial

Neste caderno (adaptado do [Projeto Pythia](https://foundations.projectpythia.org/core/xarray/enso-xarray.html)), você irá praticar o uso de várias ferramentas para examinar a temperatura da superfície do mar (SST) e explorar variações no sistema climático que ocorrem durante os eventos El Niño e La Niña. Você aprenderá a:

1. Carregar dados de Temperatura da Superfície do Mar do produto OISST
2. Mascarar dados usando `.where()`
3. Calcular climatologias e anomalias usando `.groupby()`
4. Usar `.rolling()` para calcular a média móvel
5. Calcular, normalizar e plotar o Índice Oceânico Niño

Após completar as tarefas acima, você deverá ser capaz de plotar o [Índice Oceânico Niño](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni) que se assemelha à figura abaixo. As regiões vermelha e azul correspondem às fases de El Niño e La Niña, respectivamente.

![Gráfico do índice ONI do Guia de Dados Climáticos da NCAR](https://climatedataguide.ucar.edu/sites/default/files/styles/extra_large/public/2022-03/indices_oni_2_2_lg.png)

Crédito: [NCAR](https://climatedataguide.ucar.edu/sites/default/files/styles/extra_large/public/2022-03/indices_oni_2_2_lg.png)

Crédito do Pythia: Rose, B. E. J., Kent, J., Tyle, K., Clyne, J., Banihirwe, A., Camron, D., May, R., Grover, M., Ford, R. R., Paul, K., Morley, J., Eroglu, O., Kailyn, L., & Zacharias, A. (2023). Pythia Foundations (Versão v2023.05.01) https://zenodo.org/record/8065851

# Seção 1: Introdução à Oscilação Sul do El Niño (ENSO)

A região geográfica compreendida entre **5ºN-5ºS e 190ºE-240ºE (ou 170ºW-120ºW)** é conhecida como **região Niño 3.4**. Esta região se localiza no Oceano Pacífico tropical e é comumente usada como uma métrica para determinar a fase da Oscilação Sul do El Niño (ENSO). ENSO é um padrão climático recorrente que envolve mudanças na SST no Oceano Pacífico tropical central e oriental, que possui duas fases alternadas:

- **El Niño:** a fase do ENSO caracterizada por SSTs mais quentes que a média no Oceano Pacífico tropical central e oriental, enfraquecimento dos ventos equatoriais de leste a oeste e aumento das chuvas no Pacífico tropical oriental.
- **La Niña:** a fase do ENSO que é caracterizada por SSTs mais frias que a média no Oceano Pacífico tropical central e oriental, ventos equatoriais de leste a oeste mais fortes e diminuição das chuvas no Pacífico tropical oriental.

## Seção 1.1: Processos Climáticos no Pacífico Tropical

Para entender melhor os processos do sistema climático que resultam nos eventos El Niño e La Niña, vamos primeiro considerar as condições climáticas típicas no Oceano Pacífico tropical. Os **ventos alísios** são ventos que sopram de leste a oeste ao norte e ao sul do equador (às vezes referidos como ventos "de leste" já que os ventos têm origem no leste e sopram em direção ao oeste). E como discutimos ontem, a razão pela qual os ventos alísios sopram de leste a oeste está relacionada à rotação da Terra, que faz com que os ventos no Hemisfério Norte se curvem para a direita e os ventos no Hemisfério Sul se curvem para a esquerda. Isso é conhecido como o **efeito de Coriolis**.

Se a rotação da Terra afeta o movimento do ar, você acha que também influencia o movimento da água do oceano superficial? Sim! Conforme os ventos alísios sopram sobre o Oceano Pacífico tropical, eles movem a água devido ao atrito na superfície do oceano. Mas devido ao efeito de Coriolis, a água da superfície do oceano se move para a direita da direção do vento no Hemisfério Norte e para a esquerda da direção do vento no Hemisfério Sul. No entanto, a velocidade e a direção do movimento da água mudam com a profundidade. A água da superfície do oceano se move em um ângulo em relação ao vento, e a água abaixo da superfície se move em um ângulo ligeiramente maior, e a água abaixo disso gira em um ângulo ainda maior. A direção média de toda essa água em movimento é aproximadamente um ângulo reto em relação à direção do vento. Esse valor médio é conhecido como **transporte de Ekman**. Como esse processo é impulsionado pelos ventos alísios, a intensidade desse transporte de água oceânica varia em resposta a mudanças na intensidade dos ventos alísios.

## Seção 1.2: Interações Oceano-Atmosfera Durante El Niño e La Niña

Então, como tudo isso se relaciona com El Niño e La Niña? Mudanças na intensidade dos ventos alísios do Oceano Pacífico e o impacto resultante no transporte de Ekman criam variações na SST do Pacífico tropical, o que resulta em mudanças nos padrões de circulação atmosférica e nas chuvas.

Durante um evento El Niño, **_os ventos alísios de leste são mais fracos_**. Como resultado, menos água superficial quente é transportada para o oeste via transporte de Ekman, o que causa um acúmulo de água superficial quente no Pacífico equatorial oriental. Isso cria SSTs mais quentes que a média no Oceano Pacífico equatorial oriental. A atmosfera responde a esse aquecimento com um aumento no movimento ascendente do ar e chuvas acima da média no Pacífico oriental. Em contraste, durante um evento La Niña, **_os ventos alísios de leste são mais fortes_**. Como resultado, mais água superficial quente é transportada para o oeste via transporte de Ekman, e água fria de mais profundidade no oceano sobe no Pacífico oriental durante um processo conhecido como ressurgência. Isso cria SSTs mais frias que a média no Oceano Pacífico equatorial oriental. Esse resfriamento diminui o movimento ascendente do ar no Pacífico oriental, resultando em condições mais secas que a média.

Neste tutorial, examinaremos as temperaturas de SST para explorar variações no sistema climático que ocorrem durante os eventos El Niño e La Niña. Especificamente, vamos plotar e interpretar dados de SST do OISST da região Niño 3.4.



### Fazendo o download dos dados

In [ ]:
# retrive (fetch) the SST data we are going to be working on
SST_path = DATASETS.fetch("CESM2_sst_data.nc")

# open the file we acquired with xarray
SST_data = xr.open_dataset(SST_path)

# remember that one degree spatial cell is not constant around the globe, each is a different size in square km.
# we need to account for this when taking averages for example

# fetch the weight for each grid cell
gridvars_path = DATASETS.fetch("CESM2_grid_variables.nc")

# open and save only the gridcell weights whose variable name is 'areacello'
# here the 'o' at the end refers to the area cells of the 'ocean' grid
areacello_data = xr.open_dataset(gridvars_path).areacello

# merge the SST and weights into one easy to use dataset - ds stands for dataset
ds_SST = xr.merge([SST_data, areacello_data])
ds_SST

### Carregando os arquivos e primeiros plots

In [ ]:
da_SST.plot()

### Gerando a TSM para o Niño 3.4

In [ ]:
latitude = da['lat']
weights = np.cos(np.deg2rad(latitude))
weights.name = "weights"